In [3]:
import pandas as pd

melb_data=pd.read_csv('data/melb_data_ps.csv', sep=',')

In [4]:
melb_df=melb_data.copy()

melb_df.drop(['index','Coordinates'],axis=1,inplace=True)

In [ ]:
total_rooms=melb_df['Rooms'] + melb_df['Bathroom'] + melb_df['Bedroom']
display(total_rooms)

In [ ]:
melb_df['MeanRoomsSquare']=melb_df['BuildingArea']/total_rooms
display(melb_df['MeanRoomsSquare'])

In [ ]:
diff_area=melb_df['BuildingArea']-melb_df['Landsize']
sum_area=melb_df['BuildingArea']+melb_df['Landsize']

melb_df['AreaRatio']=diff_area/sum_area
display(melb_df['AreaRatio'])

In [ ]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
display(melb_df['Date'])

In [ ]:
years_sold = melb_df['Date'].dt.year

print('Min year sold:', years_sold.min())
print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0])

In [ ]:
#Теперь попробуем понять, на какие месяцы приходится пик продаж объектов недвижимости. 
# Для этого выделим атрибут dt.month и на этот раз занесём результат в столбец MonthSale, 
# а затем найдём относительную частоту продаж для каждого месяца от общего количества продаж — 
# для этого используем метод value_counts() с параметром normalize (вывод в долях)

melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)
display(melb_df['MonthSale'].value_counts(normalize=True))


In [ ]:
#Часто бывает такая ситуация, что необходимо вычислять интервалы между двумя временными промежутками. 
# Например, можно вычислить, сколько дней прошло с 1 января 2016 года до момента продажи объекта. 
# Для этого можно просто найти разницу между датами продаж и заявленной датой, представленной в формате datetime

delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01')
#Чтобы превратить количество дней из формата интервала в формат целого числа дней, 
# можно воспользоваться аксессором dt для формата timedelta и извлечь из него атрибут days:

display(delta_days.dt.days)

In [ ]:
#Рассмотрим другой пример. Давайте создадим признак возраста объекта недвижимости в годах на момент продажи. 
# Для этого выделим из столбца с датой продажи год и вычтем из него год постройки здания. 
# Результат оформим в виде столбца AgeBuilding:
melb_df['AgeBuilding']=melb_df['Date'].dt.year-melb_df['YearBuilt']
display(melb_df['AgeBuilding'])
# удалить теперь не нужный столбец

melb_df=melb_df.drop('YearBuilt', axis=1)

In [ ]:
#Создайте в таблице melb_df признак WeekdaySale (день недели). 
# Найдите, сколько объектов недвижимости было продано в выходные (суббота и воскресенье), 
# результат занесите в переменную weekend_count. 
# В качестве ответа введите результат вывода переменной weekend_count.
melb_df['WeekdaySale']=melb_df['Date'].dt.dayofweek

weekend_count=melb_df[(melb_df['WeekdaySale']==5)].shape[0]+melb_df[(melb_df['WeekdaySale']==6)].shape[0]


display(melb_df['WeekdaySale'])

In [27]:
def get_weekend(weekday):
    if weekday ==5 or weekday==6:
        return 1
   
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
display(round(melb_df[melb_df['Weekend']==1]['Price'].mean(), 0))



1081199.0

In [ ]:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

street_types = melb_df['Address'].apply(get_street_type)
display(street_types)

street_types = street_types.apply(lambda x: 'Av' if x == 'Avenue' else x)
street_types = street_types.apply(lambda x: 'Bvd' if x == 'Boulevard' else x)
street_types = street_types.apply(lambda x: 'Pde' if x == 'Parade' else x)


print(street_types.nunique())

display(street_types.value_counts())

#Для этого к результату метода value_counts применим метод nlargest(),
# который возвращает n наибольших значений из Series. Зададим n=10, 
# т. е. мы хотим отобрать десять наиболее популярных подтипов. 
# Извлечём их названия с помощью атрибута index,
# а результат занесём в переменную popular_stypes:

popular_stypes =street_types.value_counts().nlargest(10).index
print(popular_stypes)

#введём lambda-функцию, которая будет проверять, есть ли строка x в этом перечне, 
# и, если это так, lambda-функция будет возвращать x, 
# в противном случае она будет возвращать строку 'other'. 
# Наконец, применим такую функцию к Series street_types, полученной ранее, 
# а результат определим в новый столбец таблицы StreetType

melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
display(melb_df['StreetType'])
print(melb_df['StreetType'].nunique())


In [39]:
sellers=melb_df['SellerG'].value_counts()
popular_sellers =sellers.nlargest(49).index
melb_df['Sellers'] = sellers.apply(lambda x: x if x in popular_sellers else "other")
display(melb_df['Sellers'])

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
13575    NaN
13576    NaN
13577    NaN
13578    NaN
13579    NaN
Name: Sellers, Length: 13580, dtype: object